## Install dependencies

In [1]:
!pip install -q keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.1 MB/s eta 0:00:00


## Set the backend

In this tutorial we will use Tensorflow as our backend, you can change it to "jax" or "torch" for JAX and PyTorch separately.

In [2]:
import os

# Try oicking a different value from "tensoroflow", "jax" and "torch".
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import tensorflow as tf
import keras_core as keras

import numpy as np

Using TensorFlow backend


## Use a Keras layer

Layers are callable.

In [4]:
layer = keras.layers.Dense(
    2,
    activation="relu",
    bias_initializer="zeros",
)
data = np.random.uniform(size=[3, 3])
layer(data)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.       , 0.       ],
       [0.5067259, 0.       ],
       [0.       , 0.       ]], dtype=float32)>

## Write your custom layer

In [5]:
class Linear(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=[input_dim, self.units], initializer="glorot_uniform",
        )
        self.b = self.add_weight(
            shape=[self.units,], initializer="zeros"
        )

    def call(self, inputs):
        return keras.ops.matmul(inputs, self.w) + self.b

layer = Linear(2)
layer(data)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.29882827,  0.3677824 ],
       [-0.2620599 ,  0.09132142],
       [ 0.35397828,  0.3739902 ]], dtype=float32)>

# Different ways of building the model

- Sequential API
- [**Preferred**] Functional API
- Subclassing model

In this demo, we are building a 2-class CNN image classifier.


In [6]:
# Sequential model
sequential_model = keras.Sequential([
    keras.Input([28, 28, 3]),
    keras.layers.Conv2D(8, 2),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(2),
    keras.layers.Softmax(),
])

print(sequential_model.summary())
print(sequential_model(np.random.uniform(size=[1, 28, 28, 3])))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 27, 27, 8)         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 8)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 1352)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 2)                 │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax (Softmax)               │ (None, 2)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None
tf.Tensor([[0.5182026  0.48179743]], shape=(1, 2), dtype=float32)


In [7]:
# Functional model
inputs = keras.Input([28, 28, 3])
x = keras.layers.Conv2D(8, 2)(inputs)
x = keras.layers.MaxPool2D(2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(functional_model.summary())
print(functional_model(np.random.uniform(size=[1, 28, 28, 3])))

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 3)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 8)         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 8)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_1 (Flatten)             │ (None, 1352)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 2)                 │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax_1 (Softmax)             │ (None, 2)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None
tf.Tensor([[0.6137048  0.38629526]], shape=(1, 2), dtype=float32)


Let's build a complex model with functional api.

In [ ]:
# Multi-input model.
input_1 = keras.Input([28, 28, 3])
input_2 = keras.Input([28, 28, 3])
x_1 = keras.layers.Conv2D(8, 2)(input_1)
x_2 = keras.layers.Conv2D(8, 2)(input_2)
x = keras.layers.MaxPool2D(2)(x_1 + x_2)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

multi_input_functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(multi_input_functional_model.summary())

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ input_layer_3       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 27, 27, 8) │     104 │ input_layer_2[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 27, 27, 8) │     104 │ input_layer_3[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 27, 27, 8) │       0 │ conv2d_2[0][0],      │
│                     │                   │         │ conv2d_3[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_2     │ (None, 13, 13, 8) │       0 │ add[0][0]            │
│ (MaxPooling2D)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_2 (Flatten) │ (None, 1352)      │       0 │ max_pooling2d_2[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_3 (Dense)     │ (None, 2)         │   2,706 │ flatten_2[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ input_layer_1       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax_2 (Softmax) │ (None, 2)         │       0 │ dense_3[0][0]        │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 2,914 (11.38 KB)

 Trainable params: 2,914 (11.38 KB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
# Subclassing `keras.Model`

class MyModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._conv = keras.layers.Conv2D(8, 2)
        self._max_pool = keras.layers.MaxPool2D(2)
        self._flatten = keras.layers.Flatten()
        self._dense = keras.layers.Dense(2)
        self._softmax = keras.layers.Softmax()

    def call(self, inputs):
        x = self._conv(inputs)
        x = self._max_pool(x)
        x = self._flatten(x)
        x = self._dense(x)
        return self._softmax(x)

subclass_model = MyModel()

print(subclass_model(np.random.uniform(size=[1, 28, 28, 3])))
print(subclass_model.summary())

tf.Tensor([[0.5067768  0.49322313]], shape=(1, 2), dtype=float32)


Model: "my_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ ?                         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_4 (Flatten)             │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ ?                         │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax_4 (Softmax)             │ ?                         │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None


# Models are callable.

`keras.Model` instances are also callable. You can call models as if it is a function, no matter which way you used to build the model.

In [ ]:
sample_data = np.random.uniform(size=[1, 28, 28, 3])

print(sequential_model(sample_data))
print(functional_model(sample_data))
print(subclass_model(sample_data))

[[0.72899467 0.27100533]]
[[0.18588641 0.8141136 ]]
[[0.8943892 0.1056108]]


## Models can be sliced

In [ ]:
sliced_model = keras.Model(
    inputs=functional_model.inputs,
    outputs=functional_model.get_layer("conv2d_1").output,
)
sliced_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 3)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 8)         │        104 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 104 (416.00 B)

 Trainable params: 104 (416.00 B)

 Non-trainable params: 0 (0.00 B)

# Train the model

Now we have defined the model. For actual training, we need the following pieces:
- Data
- Loss/Target function
- Optimizer

In [11]:
# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

In [12]:
# Let's use functional model defined above.
model = functional_model

In [13]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [14]:
model.fit(
    data,
    label,
    batch_size=5,
    epochs=2,
)

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.7179 - sparse_categorical_accuracy: 0.5933
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.7297 - sparse_categorical_accuracy: 0.5467


# [Reading Material] Eager mode and graph mode (demo with TF syntax)

Debug in eager mode, put actual job in graph mode. Read more [here](https://www.tensorflow.org/guide/basics#graphs_and_tffunction).

In [ ]:
# Eager mode
print(model(data[0:1, ...]))

# Graph mode
print(model.predict(data[0:1, ...]))

tf.Tensor([[0.03376885 0.9662311 ]], shape=(1, 2), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
[[0.03376885 0.9662311 ]]


In [ ]:
@tf.function(jit_compile=True)
def forward_pass(x):
    outputs = model(x)
    print(outputs)
    return outputs

print(forward_pass(data[0:1, ...]))

Tensor("functional_2_1/softmax_1_1/Softmax:0", shape=(1, 2), dtype=float32)
tf.Tensor([[0.03376885 0.9662311 ]], shape=(1, 2), dtype=float32)


In [ ]:
# You can also run `fit`, `predict` in eager mode.
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    run_eagerly=True
)

# Run `predict` in eager mode by setting `run_eagerly` above.
print(model.predict(data[0:1, ...]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[[0.03376885 0.9662311 ]]


# [Reading Material] Write your custom training loop (demo with TF syntax)

We encourage using `fit` for training/finetuning as much as possible, but it's still important to write you custom training loop. For example, if you want to debug the training loop, or have a better control over each step, you can use custom training loop.

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((data, label))
train_data = train_data.batch(5).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam(0.001)
metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(data):
    x, y = data
    with tf.GradientTape() as tape:
        outputs = model(x)
        loss = loss_fn(y, outputs)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    metric.update_state(y, outputs)

    return loss


for i, data in enumerate(train_data):
    loss = train_step(data)
    print(f"Batch {i}, loss: {loss:.3f}, accuracy: {metric.result():.3f}")

Batch 0, loss: 0.593, accuracy: 0.600
Batch 1, loss: 0.633, accuracy: 0.600
Batch 2, loss: 0.884, accuracy: 0.467
Batch 3, loss: 0.686, accuracy: 0.500


# [Reading Material] Save your model

In [ ]:
# Save the weights.
model.save_weights("dummy.weights.h5")

In [ ]:
sequential_model.load_weights("dummy.weights.h5")

In [ ]:
tf.reduce_sum(sequential_model.get_layer("conv2d").kernel - model.get_layer("conv2d_1").kernel)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

# Instruction Finetuning a Generative Model

We will show how to finetune a GPT2 model with Dolly dataset. We use GPT2 for demonstration purpose, users can switch it to other KerasNLP models.

In [15]:
!pip install -q keras-nlp datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.9 MB/s eta 0:00:00


In [16]:
import keras_nlp

gpt2_causal_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en")
gpt2_causal_lm.generate("I think basketball is fun because", max_length=50)

1042301/1042301 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       
456318/456318 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       
497986112/497986112 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


"I think basketball is fun because it is fun to watch, but it's not fun to play basketball. Basketball is fun to watch because it's not fun to play basketball, but it is fun to play basketball because it's fun to play basketball"

In [17]:
# Load Dolly2 dataset from HuggingFace
from datasets import load_dataset

dolly = load_dataset("databricks/databricks-dolly-15k", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Format data with a prompt template.

In [18]:
prompt_template = {
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{response}",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n{response}",
}

In [19]:
def format_text(data):
    if len(data["context"]) == 0:
        full_text = prompt_template["prompt_no_input"].format(
            instruction=data["instruction"],
            input=data["context"],
            response=data["response"],
        )
    else:
        full_text = prompt_template["prompt_input"].format(
            instruction=data["instruction"],
            input=data["context"],
            response=data["response"],
        )

    return full_text

In [20]:
dolly_text = []
for data in dolly:
    dolly_text.append(format_text(data))

In [21]:
dolly_tf_data = tf.data.Dataset.from_tensor_slices(dolly_text)
dolly_tf_data = dolly_tf_data.batch(4).cache().prefetch(tf.data.AUTOTUNE)

Take a look at the dataset record.

In [22]:
sample = next(iter(dolly_tf_data))
print(sample)

tf.Tensor(
[b"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhen did Virgin Australia start operating?\n\n### Input:\nVirgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.\n\n### Response:\nVirgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route."
 b'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n

Take a preview of what the processed training data looks like.

In [23]:
gpt2_causal_lm.preprocessor(sample)

({'token_ids': <tf.Tensor: shape=(4, 1024), dtype=int32, numpy=
  array([[50256, 21106,   318, ...,     0,     0,     0],
         [50256, 21106,   318, ...,     0,     0,     0],
         [50256, 21106,   318, ...,     0,     0,     0],
         [50256, 21106,   318, ...,     0,     0,     0]], dtype=int32)>,
  'padding_mask': <tf.Tensor: shape=(4, 1024), dtype=bool, numpy=
  array([[ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False]])>},
 <tf.Tensor: shape=(4, 1024), dtype=int32, numpy=
 array([[21106,   318,   281, ...,     0,     0,     0],
        [21106,   318,   281, ...,     0,     0,     0],
        [21106,   318,   281, ...,     0,     0,     0],
        [21106,   318,   281, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(4, 1024), dtype=bool, numpy=
 array([[ True,  True,  True, ..., False, F

In [24]:
num_epochs = 2
lr = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=dolly_tf_data.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
gpt2_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(lr, weight_decay=0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy(),]
)

# Use MLflow to track your experiments

MLflow is a handy tool for managing the lifecycle of your ML experiments. Here we show how to use it for experiment tracking purposes.

If you have not, please register an account of [Databricks community edition](https://www.databricks.com/try-databricks#account). It should take no longer than 1min to register. Databricks CE (community edition) is a free platform for users to try out Databricks features. For this guide, we need the ML experiment dashboard for us to track our training progress.

In [25]:
!pip install -q git+https://github.com/mlflow/mlflow.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


After you have sucessfully registered an account, all you need to do is to run the command below to connect from Google Colab to your Databricks account. You will need to enter following information at prompt:
- **Databricks Host**: https://community.cloud.databricks.com/
- **Username**: your signed up email
- **Password**: your password

In [26]:
!databricks configure

Databricks Host (should begin with https://): https://community.cloud.databricks.com/
Username: qianchen94era@gmail.com
Password: 
Repeat for confirmation: 


In [27]:
import mlflow

mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/2023-ghc-keras-demo")

with mlflow.start_run() as run:
    mlflow_callback = mlflow.keras_core.MLflowCallback(
        run,
        log_every_epoch=False,
        log_every_n_steps=20,
    )
    gpt2_causal_lm.fit(
        dolly_tf_data.take(100),
        epochs=num_epochs,
        callbacks=[mlflow_callback],
    )

Epoch 1/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - loss: 0.5444 - sparse_categorical_accuracy: 0.0931
Epoch 2/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - loss: 0.4385 - sparse_categorical_accuracy: 0.1078


In [ ]:
gpt2_causal_lm.generate("I think basketball is fun because", max_length=50)

"I think basketball is fun because it's fun, but it's also hard to learn from. You can't really learn how to play basketball from the outside. You have to learn how to play the game. You have to learn how to play"